In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-07-01


Found rundate 2023-06-30 10:00:02 < 2023-07-01 at commit 0b5168de; returning commit d4b534b


'2023-07-01 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-07-01 10:00:03, NJSP was reporting 268 YTD deaths
Current YTD Deaths (2024-07-01 10:00:03-04:00): 313
Previous year YTD Deaths (adjusted; 2023-07-01 10:00:03): 268.74132377178665
Projected 2024 total: 680.0993174295734
16.5% change


FAUQStats cache miss: 2023, Sat Jul 01 10:00:03 EDT 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,NaN,Mulberry St,1.0,0.0,0.0,1.0,0.0,NaN,Mulberry St,2023-01-01 01:49:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,322,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,NaN,South Randolphville Rd,1.0,1.0,0.0,0.0,0.0,0.0,South Randolphville Rd,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,NaN,Outwater Ln,1.0,0.0,0.0,1.0,0.0,0.0,Outwater Ln,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,NaN,Washington St,1.0,0.0,0.0,1.0,0.0,NaN,Washington St,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12535,11,Mercer,1113,West Windsor Twp,535,County 535,1.0,0.0,1.0,0.0,0.0,NaN,NaN,2023-06-26 22:40:00-04:00
12533,16,Passaic,1614,Wayne Twsp,80,Interstate 80 MP 53,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2023-06-27 03:35:00-04:00
12536,08,Gloucester,0809,Logan Twsp,130,State Highway 130,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2023-06-28 14:42:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,14,1,11,2,28
Bergen,9,3,9,0,20
Burlington,17,3,2,1,23
Camden,11,3,10,0,24
Cape May,1,0,0,0,1
Cumberland,8,2,2,0,11
Essex,8,3,11,0,22
Gloucester,7,2,0,0,9
Hudson,2,2,1,0,4


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

49.8% through the year, 50.2% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,8,18,14
Bergen,driver,10,20,9
Burlington,driver,10,26,17
Camden,driver,9,18,11
Cape May,driver,2,4,1
...,...,...,...,...
Salem,crashes,4,11,7
Somerset,crashes,6,21,7
Sussex,crashes,1,6,6


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,60,4,28,5,24
Bergen,40,1,19,9,15
Burlington,52,3,39,6,4
Camden,56,3,21,7,27
Cape May,4,0,3,1,1
Cumberland,21,0,16,4,3
Essex,55,1,17,5,36
Gloucester,20,1,14,3,2
Hudson,11,1,6,5,3


In [9]:
projected.to_csv(PROJECTED_CSV)